# TMPL Cheatsheet
This notebook illustrates how to create TMPL classes, combine them into a test sequence, run the sequence and look at the results.

In [1]:
# Use this to import tmpl if it has not been installed by pip
import numpy as np
import set_path_for_notebooks
import tmpl

/home/redlegjed/Documents/Python/Projects/test_measure_process_lib


In [2]:
# Use this if tmpl has been installed by pip
import numpy as np
import tmpl

# Create TMPL classes
In the following sections the core TMPL clases: *SetupCondition*,*Measurement* and *TestManager* will be created.

## Create *SetupCondition* classes
*SetupCondition* classes require two mandatory properties: *actual* and *setpoint* and usually an *initialise()* method to define the default values.

The code below shows the standard way to define the properties and methods. The *actual* and *setpoint* properties are created as functions with the *property* decorator. This is because they are usually used to set parameters on lab instrumentation. However from the test sequence point of view we just want to set or query the value which is easier from a property interface.

Once a *SetupCondition* object is created it can be used in the following way:

```python

# Read current value of condition
condition.actual

# Read current setpoint of condition
condition.setpoint

# Set new value of condition
condition.setpoint = new_value

```

The following code creates two *SetupCondition* classes for temperature and humidity. As this is an example we are not connected to any lab equipment so the setpoint is being faked by an internal variable *_setpoint*.

In [3]:

class TemperatureConditions(tmpl.AbstractSetupConditions):
    """
    Temperature setpoint condition

    """
    name = 'temperature_degC'

    def initialise(self):
        """
        Initialise default values and any other setup
        """

        # Set default values
        self.values = [25,35,45]

        # Setpoint
        self._setpoint = 25

    @property
    def actual(self):
        return self.resistor.temperature_degC

    @property
    def setpoint(self):
        return self._setpoint

    @setpoint.setter
    def setpoint(self,value):
        self.log(f'Setpoint = {value} degC')
        self._setpoint = value
        # Use set_temperature, which is automatically available
        # from resources
        return self.set_temperature(self.resistor,value)



class HumidityConditions(tmpl.AbstractSetupConditions):
    """
    Humidity setpoint condition

    """
    name = 'humidity_pc'

    def initialise(self):
        """
        Initialise default values and any other setup
        """
        # Set default values
        self.values = [55,60,70]

        # Setpoint
        self._setpoint = 45

    @property
    def actual(self):
        return self.resistor.humidity_pc

    @property
    def setpoint(self):
        return self._setpoint

    @setpoint.setter
    def setpoint(self,value):
        self.log(f'Setpoint = {value} %')
        self._setpoint = value
        # Use set_humidity, which is automatically available
        # from resources
        return self.set_humidity(self.resistor,value)


## Create *Measurement* objects
*Measurement* objects have one mandatory method: *meas_sequence()* which is where the top level measurement code should go. When a test sequence is run it will run *meas_sequence* for all enabled *Measurement* objects.

*Measurement* object can also have optional methods:
* *initialise()* : for setting default configuration settings
* *process()* : for processing measurement data, this will be run after *meas_sequence()* if defined.

These are the methods that the TMPL machinery will recognise. The user may add any other methods or properties just like a normal class to use for breaking up the measurement code.

The code below defines a set of *Measurement* classes for measuring a resistor. The classes are:

* *VoltageSweeper* : This is the main measurement of the sequence. It contains a *process()* method and another convenience method for converting units that uses *services* that will be defined later.
* *Stabilise* : A simple *Measurement* class for waiting for stabilisation that only runs when the temperature has changed
* *TurnOn* : A class for starting up the equipment that only runs at the startup stage of the sequence
* *TurnOff* : A class for shutting down the equipment that only runs at the end of the sequence.

In [4]:

class VoltageSweeper(tmpl.AbstractMeasurement):
    """
    Example of a Measurement that adds its own coordinates and has
    a process method

    Measurement method:

    * Sweep voltage
    * Measure current at each voltage step
    * Process voltage and current to calculate resistances
    """
    name = 'VoltageSweep' # Optional, by default this will be class name

    def initialise(self):

        # Set up the voltage values to sweep over
        self.config.voltage_sweep = np.linspace(0,1,10)
        
    def meas_sequence(self):
        
        #  Do the measurement
        
        current = np.zeros(self.config.voltage_sweep.shape)

        for index,V in enumerate(self.config.voltage_sweep):
            # Set voltage
            self.voltage_supply.set_voltage(V,self.resistor)

            # Measure current
            current[index] = self.resistor.current_A

        
        # Store the data
        self.store_coords('swp_voltage',self.config.voltage_sweep)
        self.store_data_var('current_A',current,coords=['swp_voltage'])

        # Debug point
        self.log('finished sweep')


    @tmpl.with_results(data_vars=['current_A'])
    def process(self):

        # Get measurement data for current set of conditions
        ds = self.current_results

        # Fit a line to current vs voltage
        p = ds.current_A.polyfit('swp_voltage',1)

        # Get resistance from slope of line
        resistance_ohms = p.polyfit_coefficients.sel(degree=1).values

        # Store data into self.ds_results
        self.store_data_var('resistance_ohms',[resistance_ohms])

        self.convert_units()


    @tmpl.with_services(['Amps_to_mA'])
    def convert_units(self):
        """
        Example of using services

        This method uses the service Amps_to_mA which is defined in the 
        main test sequence
        """

        current = self.current_results.current_A.values

        # Use services function to convert to mA
        self.store_data_var('current_mA',self.services.Amps_to_mA(current),
                        coords=['swp_voltage'])



# Measurement class that runs every time the temperature changes
class Stabilise(tmpl.AbstractMeasurement):
    """
    Wait for stabilisation

    """

    def initialise(self):
        self.run_on_setup('temperature_degC')

    def meas_sequence(self):
        self.log('Stabilising')


# Measurement classes that run at start and end of test sequence
class TurnOn(tmpl.AbstractMeasurement):
    """
    Turn on all equipment

    """

    def initialise(self):
        self.run_on_startup(True)


    def meas_sequence(self):
        self.log('TurnOn measurement')


class TurnOff(tmpl.AbstractMeasurement):
    """
    Turn off all equipment

    """

    def initialise(self):
        self.run_on_teardown(True)


    def meas_sequence(self):
        self.log('TurnOff measurement')



## Create *TestManager* classes

*TestManager* classes define a specific sequence of *SetupConditions* and *Measurements*. They are very simple classes that require two mandatory methods to be defined by the user:

* *define_setup_condition()*: Add all setup conditions to sequence in the order that they will be set.
* *define _measurements()*: Add all measurements to sequence in the order of execution

There are two optional methods that are normally defined as well:

* *initialise()* : Usually used to define default configuration settings
* *define_services()* : Add globally available *services*. These are functions that any *SetupCondition* or *Measurement* object can access through their internal property *.services*.

The following code creates a single *TestManager* class that combines the *SetupConditions* and *Measurements* defined above into a test sequence.

In [5]:

class ExampleTestSequence(tmpl.AbstractTestManager):
    """
    Example test sequence

    Runs a dummy measurement sequence over temperature and humidity conditions.

    Measurement sequence is:

    * Turn on equipment
    * Wait for stabilisation
    * Run a voltage sweep
    * Turn off equipment

    """
    name = 'ExampleResistorTest'

    def define_setup_conditions(self):
        """
        Add the setup conditions here in the order that they should be set
        """

        self.add_setup_condition(TemperatureConditions)
        self.add_setup_condition(HumidityConditions)


    def define_measurements(self):
        """
        Add measurements here in the order of execution
        """

        # Setup links to all the measurements
        self.add_measurement(TurnOn)
        self.add_measurement(Stabilise)
        self.add_measurement(VoltageSweeper)
        self.add_measurement(TurnOff)

        # Go here if errors occur
        # self.add_measurement(HandleError)


    def define_services(self) -> None:
        """
        Add globally available functions to services property.
        This will get copied to all SetupConditions and Measurements
        """

        self.services.Amps_to_mA = lambda Amps: Amps*1000

# Create a *TestManager* object for running a test sequence
We now need to instantiate the *TestManager* object in order to run the sequence.

Usually this requires 'resources' in the shape of test equipment interfaces. For this example we will cheat and use dummy equipment from the *example_resistor_test.py* module:

In [6]:
from example_resistor_test import ResistorModel, set_temperature, set_humidity,VoltageSupply


Now we create a *resources* input for the *TestManager* as a dictionary. 

In [7]:
res1 = ResistorModel(100,tolerance_pc=1.0)

# Setup resources
resources = {
    'set_temperature':set_temperature,
    'set_humidity': set_humidity,
    'voltage_supply':VoltageSupply(),
    'resistor':res1,
    }

Next instantiate a *TestManager* object using the *ExampleTestSequence* class:

In [8]:
test = ExampleTestSequence(resources)

## Displaying information about test sequence
We can display useful information about the *TestManager* object just from a Jupyter cell like this:

In [9]:
test

TestManager[ExampleResistorTest]
* Conditions:
	Iteration
	temperature_degC
	humidity_pc
* Measurements:
	Timestamp
	TurnOn
	Stabilise
	VoltageSweep
	TurnOff

The *TestManager* object will display which *Measurements* and *SetupConditions* are being used.

### Sequence running order
If we want to know how the test sequence will run we use the *df_running_order* property to display a table show when conditions are set and when measurements are made:

In [10]:
test.df_running_order

[SEQ] ExampleTestSequence       | Generating the sequence running order
[SEQ] ExampleTestSequence       | 	Running order done


,Operation,Label,Iteration,temperature_degC,humidity_pc
0,MEASUREMENT,Timestamp,None,NaN,NaN
1,MEASUREMENT,TurnOn,None,NaN,NaN
2,CONDITION,temperature_degC,None,25.0,NaN
3,MEASUREMENT,Stabilise,None,25.0,NaN
4,CONDITION,humidity_pc,None,NaN,55.0
5,MEASUREMENT,VoltageSweep,None,25.0,55.0
6,CONDITION,humidity_pc,None,NaN,60.0
7,MEASUREMENT,VoltageSweep,None,25.0,60.0
8,CONDITION,humidity_pc,None,NaN,70.0
9,MEASUREMENT,VoltageSweep,None,25.0,70.0


This table has two columns that are always the same: *Operation* and *Label*. *Operation* states whether a condition or measurement is being performed. *Label* gives the name of the condition or measurement.

The other columns are for the values of the *SetupConditions*, one column for each condition. The contents of these columns shows when in the sequence the conditions are changed. 

In our example above temperature has values 25,35 & 45 and humidity has 55,60 & 70. Each time the temperature is changed all three humidity levels are set and the *VoltageSweep* is measured. The temperature and humidity levels are those defined in the *initialise()* methods of the *SetupCondition* classes. The order in which they are set is because in *ExampleTestSequence*, *define_setup_condition()* temperature is added before humidity so it is the first to be set.

The *Iteration* column is a *SetupCondition* which TMPL adds by default when the *TestManager* is instantiated. It is intended for repeating whole test sequences. By default it is disabled. We will cover it later.

# Running the test sequence
Once the classes have been defined and the running order looks correct, the test sequence can be run simply using the *run()* method. This will generate a printout as it goes through the sequence indicating which conditions and measurements are executing.

In [11]:
test.run()

[SEQ] ExampleTestSequence       | <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
[SEQ] ExampleTestSequence       | Running ExampleResistorTest
[SEQ] ExampleTestSequence       | Generating the sequence running order
[SEQ] ExampleTestSequence       | 	Running order done
[M] Timestamp                 | <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
[M] Timestamp                 | Running Timestamp
[M] Timestamp                 | Timestamp	Time taken: 0.002 s 
[M] Timestamp                 | >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[M] TurnOn                    | <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
[M] TurnOn                    | Running TurnOn
[M] TurnOn                    | TurnOn measurement
[M] TurnOn                    | TurnOn	Time taken: 0.001 s 
[M] TurnOn                    | >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

------------------------------------------------------------
[C] TemperatureConditions     | Setpoint = 25 degC
[M] Stabilise                 | <<<<<<<<<<<<<<<<<<<<<<<<<<<<

# Looking at test results

Assuming that the measurements have been written to use the *store_coords()* and *store_data_vars()* method for storing data then the results data should be stored in an [xarray](https://docs.xarray.dev/en/stable/index.html) Dataset. This can be viewed using the *ds_results* property of the *TestManager* object.

In [12]:
test.ds_results

<xarray.Dataset>
Dimensions:           (default: 1, timestamp: 1, temperature_degC: 3,
                       humidity_pc: 3, swp_voltage: 10)
Coordinates:
  * default           (default) int64 0
  * timestamp         (timestamp) <U19 '2023-01-08 14h40m32'
  * temperature_degC  (temperature_degC) int64 25 35 45
  * humidity_pc       (humidity_pc) int64 55 60 70
  * swp_voltage       (swp_voltage) float64 0.0 0.1111 0.2222 ... 0.8889 1.0
Data variables:
    current_A         (temperature_degC, humidity_pc, swp_voltage) float64 0....
    resistance_ohms   (temperature_degC, humidity_pc) float64 0.007902 ... 0....
    current_mA        (temperature_degC, humidity_pc, swp_voltage) float64 0....

Looking at the *ds_results* Dataset the *SetupConditions* become [coordinates](https://docs.xarray.dev/en/stable/user-guide/data-structures.html#coordinates). The measured values become [data variables](https://docs.xarray.dev/en/stable/user-guide/data-structures.html#dataset) in xarray terminology.

The dependencies of each measured *Data variable* is shown in the Dataset. Note also that there is a *coordinate* *swp_voltage* that was not defined as a *SetupCondition* but from within the *VoltageSweeper* class using the *store_coords()* method.

## Storing data
Now that the measurements have been taken and are stored in the *ds_results* property they can be saved.

*TestManagers* have a *save()* method that allows *ds_results* to be stored to a JSON file

In [13]:
test.save('example_measured_data.json')

There is also a *load()* method that can be used to reload results from a previous sequence into a *TestManager*. 

This is useful for debugging processing code offline. In this case a new *TestManager* object can be created for the purpose. If it is offline from the instruments then the 'resources' argument can be passed as an empty dict

In [14]:
test_debug = ExampleTestSequence({})
test_debug.load('example_measured_data.json')

# Customising test sequences

When the *TestManager*, *SetupCondition* and *Measurement* classes are defined it is usual to give them default configuration settings. However once the class objects have been created these configuration settings can be changed to alter the scope of the test sequence.

**WARNING** we are about to change the test sequence settings, if you want to re-run the code above make sure to re-instantiate the *test* object.

## Customising *SetupConditions*
During the test sequence *SetupConditions* will iterate through the contents of their *values* property, which is a list. Here it is for temperature in our example:

(Note how to access the *SetupConditions* object through the *TestManager* *conditions* property)

In [15]:
test.conditions.temperature_degC.values

[25, 35, 45]

If we only want to measure one temperature, we can change the value in the list and then check the running order to see if the change worked:

In [16]:
# Change setpoints of condition
test.conditions.temperature_degC.values = [40]

# Check the running order
test.df_running_order

[SEQ] ExampleTestSequence       | Generating the sequence running order
[SEQ] ExampleTestSequence       | 	Running order done


,Operation,Label,Iteration,temperature_degC,humidity_pc
0,MEASUREMENT,Timestamp,None,NaN,NaN
1,MEASUREMENT,TurnOn,None,NaN,NaN
2,CONDITION,temperature_degC,None,40.0,NaN
3,MEASUREMENT,Stabilise,None,40.0,NaN
4,CONDITION,humidity_pc,None,NaN,55.0
5,MEASUREMENT,VoltageSweep,None,40.0,55.0
6,CONDITION,humidity_pc,None,NaN,60.0
7,MEASUREMENT,VoltageSweep,None,40.0,60.0
8,CONDITION,humidity_pc,None,NaN,70.0
9,MEASUREMENT,VoltageSweep,None,40.0,70.0


Now only one temperature is being set, which reduces the size of the running order table.

## Customising measurements
Now we might want to change the settings in the *VoltageSweeper* measurement. By default it will sweep 10 voltages in the range 0 to 1V. 

We can see this by checking the maximum value of the *swp_voltage* coordinate in *ds_results*:

In [17]:
test.ds_results.swp_voltage.max()

<xarray.DataArray 'swp_voltage' ()>
array(1.)

Let's increase the range to 2V by editing the setting in the *config* property.

We can access *VoltageSweeper* through the *meas* property of the *TestManager*

In [18]:
test.meas.VoltageSweep.config.voltage_sweep = np.linspace(0,2,10)

Now we can re-run the measurement and check *swp_voltage* again

In [19]:
test.run()

[SEQ] ExampleTestSequence       | <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
[SEQ] ExampleTestSequence       | Running ExampleResistorTest
[SEQ] ExampleTestSequence       | Generating the sequence running order
[SEQ] ExampleTestSequence       | 	Running order done
[M] Timestamp                 | <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
[M] Timestamp                 | Running Timestamp
[M] Timestamp                 | Timestamp	Time taken: 0.001 s 
[M] Timestamp                 | >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[M] TurnOn                    | <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
[M] TurnOn                    | Running TurnOn
[M] TurnOn                    | TurnOn measurement
[M] TurnOn                    | TurnOn	Time taken: 0.001 s 
[M] TurnOn                    | >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

------------------------------------------------------------
[C] TemperatureConditions     | Setpoint = 40 degC
[M] Stabilise                 | <<<<<<<<<<<<<<<<<<<<<<<<<<<<

In [20]:
test.ds_results.swp_voltage.max()

<xarray.DataArray 'swp_voltage' ()>
array(2.)

Now we can see the results of our changes: the temperature was only set to 40degC and the *swp_voltage* now goes out to 2V. We can see all this just by looking at *ds_results*:

In [21]:
test.ds_results

<xarray.Dataset>
Dimensions:           (default: 1, timestamp: 1, temperature_degC: 1,
                       humidity_pc: 3, swp_voltage: 10)
Coordinates:
  * default           (default) int64 0
  * timestamp         (timestamp) <U19 '2023-01-08 14h40m33'
  * temperature_degC  (temperature_degC) int64 40
  * humidity_pc       (humidity_pc) int64 55 60 70
  * swp_voltage       (swp_voltage) float64 0.0 0.2222 0.4444 ... 1.778 2.0
Data variables:
    current_A         (temperature_degC, humidity_pc, swp_voltage) float64 0....
    resistance_ohms   (temperature_degC, humidity_pc) float64 0.007065 ... 0....
    current_mA        (temperature_degC, humidity_pc, swp_voltage) float64 0....

# Repeatability

We previously noted that there is an automatically generated *SetupCondition* called *Iteration* in the running order table. It is there so that you can repeat a test sequence many times without having to add extra loops to your code. 

Let's try repeating the measurement 3 times.

Before running, we will reduce the humidity to one value to reduce the amount of printing.

In [22]:
test.conditions.humidity_pc.values = [50]

Now our basic measurement running order is:

In [23]:
test.df_running_order

[SEQ] ExampleTestSequence       | Generating the sequence running order
[SEQ] ExampleTestSequence       | 	Running order done


,Operation,Label,Iteration,temperature_degC,humidity_pc
0,MEASUREMENT,Timestamp,None,NaN,NaN
1,MEASUREMENT,TurnOn,None,NaN,NaN
2,CONDITION,temperature_degC,None,40.0,NaN
3,MEASUREMENT,Stabilise,None,40.0,NaN
4,CONDITION,humidity_pc,None,NaN,50.0
5,MEASUREMENT,VoltageSweep,None,40.0,50.0
6,MEASUREMENT,TurnOff,None,NaN,NaN


We will now enable the *Iteration* condition and set it to run over 3 values, which will be labelled 1,2 & 3.

In [24]:
# Enable Iteration condition
test.conditions.Iteration.enable = True

# Set number of iterations as a list
test.conditions.Iteration.values = [1,2,3]

# Note: could also use list comprehension, especially for large number
# test.conditions.Iteration.values = [n for n in range(25)]


Now let's take a look at the running order:

In [25]:
test.df_running_order

[SEQ] ExampleTestSequence       | Generating the sequence running order
[SEQ] ExampleTestSequence       | 	Running order done


,Operation,Label,Iteration,temperature_degC,humidity_pc
0,MEASUREMENT,Timestamp,NaN,NaN,NaN
1,MEASUREMENT,TurnOn,NaN,NaN,NaN
2,CONDITION,Iteration,1.0,NaN,NaN
3,CONDITION,temperature_degC,NaN,40.0,NaN
4,MEASUREMENT,Stabilise,1.0,40.0,NaN
5,CONDITION,humidity_pc,NaN,NaN,50.0
6,MEASUREMENT,VoltageSweep,1.0,40.0,50.0
7,CONDITION,Iteration,2.0,NaN,NaN
8,MEASUREMENT,VoltageSweep,2.0,40.0,50.0
9,CONDITION,Iteration,3.0,NaN,NaN


The *Iteration* column is now changing. Every time it changes the core measurements and setup conditions are executed. Only *TurnOn* and *TurnOff* are not affected because they were tagged to run at startup and teardown stages.

Running the test again will gives us result over the three iterations:

In [26]:
test.run()

[SEQ] ExampleTestSequence       | <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
[SEQ] ExampleTestSequence       | Running ExampleResistorTest
[SEQ] ExampleTestSequence       | Generating the sequence running order
[SEQ] ExampleTestSequence       | 	Running order done
[M] Timestamp                 | <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
[M] Timestamp                 | Running Timestamp
[M] Timestamp                 | Timestamp	Time taken: 0.001 s 
[M] Timestamp                 | >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[M] TurnOn                    | <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
[M] TurnOn                    | Running TurnOn
[M] TurnOn                    | TurnOn measurement
[M] TurnOn                    | TurnOn	Time taken: 0.000 s 
[M] TurnOn                    | >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

------------------------------------------------------------
[C] Iteration                 | Iteration = 1 
------------------------------------------------------------
[C]

Looking at the results:

In [27]:
test.ds_results

<xarray.Dataset>
Dimensions:           (default: 1, timestamp: 1, Iteration: 3,
                       temperature_degC: 1, humidity_pc: 1, swp_voltage: 10)
Coordinates:
  * default           (default) int64 0
  * timestamp         (timestamp) <U19 '2023-01-08 14h40m34'
  * Iteration         (Iteration) int64 1 2 3
  * temperature_degC  (temperature_degC) int64 40
  * humidity_pc       (humidity_pc) int64 50
  * swp_voltage       (swp_voltage) float64 0.0 0.2222 0.4444 ... 1.778 2.0
Data variables:
    current_A         (Iteration, temperature_degC, humidity_pc, swp_voltage) float64 ...
    resistance_ohms   (Iteration, temperature_degC, humidity_pc) float64 0.00...
    current_mA        (Iteration, temperature_degC, humidity_pc, swp_voltage) float64 ...

Note that *Iteration* has appeared as a coordinate with values 1,2,3 and the *Data variables* all have it as a dependency.

We can use *xarrays* tools to analyse our data over iterations:

In [28]:
# Find average resistance over all iterations
test.ds_results.resistance_ohms.mean()

<xarray.DataArray 'resistance_ohms' ()>
array(0.00706714)

Again the new *ds_results* Dataset can be saved using *test.save(...)* for further analysis.

# Disabling conditions

Setup conditions can be enabled/disabled using the *enable* property

In [30]:
# Disable humidity and display new running order
test.conditions.humidity_pc.enable = False
test.df_running_order

[SEQ] ExampleTestSequence       | Generating the sequence running order
[SEQ] ExampleTestSequence       | 	Running order done


,Operation,Label,Iteration,temperature_degC,humidity_pc
0,MEASUREMENT,Timestamp,NaN,NaN,None
1,MEASUREMENT,TurnOn,NaN,NaN,None
2,CONDITION,Iteration,1.0,NaN,None
3,CONDITION,temperature_degC,NaN,40.0,None
4,MEASUREMENT,Stabilise,1.0,40.0,None
5,MEASUREMENT,VoltageSweep,1.0,40.0,None
6,CONDITION,Iteration,2.0,NaN,None
7,MEASUREMENT,VoltageSweep,2.0,40.0,None
8,CONDITION,Iteration,3.0,NaN,None
9,MEASUREMENT,VoltageSweep,3.0,40.0,None
